In [1]:
from kaggle_environments import make
from scipy.stats import norm

import hyperopt
import numpy as np
import os
if not os.path.exists('tmp'):
    os.mkdir('tmp')
    
base_path = '/Users/sergmiller/Documents/my/bandits'

Loading environment football failed: No module named 'gfootball'


In [2]:
from utils.bandits import Agent, init_agent, \
    compare, gittins_bb_delta, init_template, neural, gittins_with_random, bench

In [3]:
from importlib import reload
import utils
reload(utils)
reload(utils.bandits)
from utils.bandits import exact_gittins

In [4]:
compare(Agent(text=utils.bandits.exact_gittins), Agent(text=gittins_with_random))

(1.4968878586693142e-05,
 42.4,
 30.971599894096528,
 array([687., 657., 613., 639., 716., 613., 606., 654., 619., 591.]),
 array([612., 592., 529., 616., 688., 623., 561., 614., 544., 592.]))

In [5]:
bench(Agent(text=utils.bandits.exact_gittins))

['gittins', (4.2410388356091665e-07, 45.6, 28.510348998214667)]
['gittins_with_random', (0.0018946078240590072, 33.7, 34.30757933751666)]
['gittins_with_count_my', (0.5443322947442247, 5.3, 27.64434842784326)]
['gittins_with_count_rival_drift', (2.069588791130939e-06, 28.8, 19.18749592833833)]
['gittins_with_my_and_count_rival_drift', (0.8731023219367382, 1.0, 19.79898987322333)]
['gittins_bb', (0.00027884018531413636, 33.2, 28.88875213642846)]
['gittins_bb_delta', (0.7698044398391066, 3.6, 38.902956185873585)]
['softmax_ucb', (7.928191572141328e-06, 37.3, 26.404734423962683)]
['multiarmed_bandit_agent', (0.13655970869685635, 10.5, 22.303587155433092)]
['upper_confidence', (4.963109977336576e-61, 147.8, 28.357714999625763)]
['ucb_decay', (7.461902643596642e-07, 39.3, 25.111949346874685)]
['bayesian_ucb', (0.6950011664656811, -3.3, 26.615972648017205)]
['thompson', (0.25003483765841505, -10.8, 29.691076100404306)]
['neural', (0.237276505928668, -9.6, 25.687350972803717)]
<string>:82: Ru

[['gittins', (4.2410388356091665e-07, 45.6, 28.510348998214667)],
 ['gittins_with_random', (0.0018946078240590072, 33.7, 34.30757933751666)],
 ['gittins_with_count_my', (0.5443322947442247, 5.3, 27.64434842784326)],
 ['gittins_with_count_rival_drift',
  (2.069588791130939e-06, 28.8, 19.18749592833833)],
 ['gittins_with_my_and_count_rival_drift',
  (0.8731023219367382, 1.0, 19.79898987322333)],
 ['gittins_bb', (0.00027884018531413636, 33.2, 28.88875213642846)],
 ['gittins_bb_delta', (0.7698044398391066, 3.6, 38.902956185873585)],
 ['softmax_ucb', (7.928191572141328e-06, 37.3, 26.404734423962683)],
 ['multiarmed_bandit_agent', (0.13655970869685635, 10.5, 22.303587155433092)],
 ['upper_confidence', (4.963109977336576e-61, 147.8, 28.357714999625763)],
 ['ucb_decay', (7.461902643596642e-07, 39.3, 25.111949346874685)],
 ['bayesian_ucb', (0.6950011664656811, -3.3, 26.615972648017205)],
 ['thompson', (0.25003483765841505, -10.8, 29.691076100404306)],
 ['neural', (0.237276505928668, -9.6, 25.68

In [7]:
compare(
    Agent(text=utils.bandits.exact_gittins_with_my_and_count_rival_drift.format("decay=0.03")),
    Agent(file='/Users/sergmiller/Documents/my/bandits/kernels/thompson.py')
)

(0.07719266577147046,
 -15.1,
 27.020177645604036,
 array([645., 658., 621., 682., 519., 656., 677., 669., 608., 678.]),
 array([644., 643., 675., 652., 576., 664., 717., 673., 629., 691.]))

In [8]:
compare(
    Agent(text=utils.bandits.exact_gittins_with_my_and_count_rival_drift.format("decay=0.03")),
    Agent(text=neural)
)

(0.3760897672331939,
 9.4,
 33.583329197683774,
 array([599., 605., 599., 611., 630., 703., 605., 666., 683., 571.]),
 array([600., 581., 608., 654., 635., 619., 610., 643., 641., 587.]))

In [9]:
def f(input):
    print(input)
    exact_gittins_custom = init_template(utils.bandits.exact_gittins_with_my_and_count_rival_drift, input)
    res = compare(Agent(text=utils.bandits.exact_gittins), Agent(text=exact_gittins_custom), T=10)[:3]
    print(res[1] + res[2], res)
    return res[1] + res[2]  # since res[1] < 0

In [14]:
search_space = {
  "alpha": hyperopt.hp.uniform('alpha', 0.05, 0.95),
  "distortion_horizon": hyperopt.hp.uniform('distortion_horizon', 1.05, 10),
  "p": hyperopt.hp.uniform('p', 0.05, 0.95),
  "q": hyperopt.hp.uniform('q', 0.05, 0.95),
  "eps": hyperopt.hp.uniform('eps', 0, 1e-1),
  "rival_drift": hyperopt.hp.normal('rival_drift', -1e-3, 1e-3),
}

# trials = hyperopt.Trials()
best = hyperopt.fmin(
  fn=f,
  space=search_space,
  algo=hyperopt.tpe.suggest,
  max_evals=400,
  verbose=True,
  trials=trials)
print(best)



{'alpha': 0.327577676428244, 'distortion_horizon': 3.879261305736294, 'eps': 0.028575954636710012, 'p': 0.88266086119104, 'q': 0.6670363553220694, 'rival_drift': -0.0003661796251315407}
27.049011281025138                                       
(0.9525096856129976, 0.5, 26.549011281025138)            
{'alpha': 0.1063059276316673, 'distortion_horizon': 2.639560096248874, 'eps': 0.04070996045266857, 'p': 0.6917154876557291, 'q': 0.5783025261142943, 'rival_drift': -0.000723411924183151}
45.12066088834041                                                                  
(0.014260278374446362, 19.7, 25.420660888340414)                                   
{'alpha': 0.35384433588073994, 'distortion_horizon': 8.017275395960773, 'eps': 0.03699691011199045, 'p': 0.5897365423487383, 'q': 0.4769531643506638, 'rival_drift': -0.0015535799028164343}
22.753540961133073                                                                 
(0.9042277912861212, -0.9, 23.65354096113307)                         

KeyboardInterrupt: 

In [ ]:
{'alpha': 0.4685023287045787, 'distortion_horizon': 1.3854054259517206, 'eps': 0.031610535865854904, 'p': 0.519036315706196, 'q': 0.8842598985284099, 'rival_drift': 0.0035039229431558017}
-2.5916690765377                                                                  
(0.00019065620309177938, -17.0, 14.4083309234623)  